In [1]:
# I will import libs

In [4]:
import dash
from dash import dcc, html
import pandas as pd
import requests
import plotly.graph_objs as go
from dash.dependencies import Input, Output

In [6]:
# I will design and starting the web app

In [8]:
app = dash.Dash(__name__)

app.layout = html.Div([
    html.H1("Real-Time Stock Price Dashboard"),
    dcc.Dropdown(
        id='stock-dropdown',
        options=[
            {'label': 'Apple', 'value': 'AAPL'},
            {'label': 'Google', 'value': 'GOOGL'},
            {'label': 'Amazon', 'value': 'AMZN'},
            {'label': 'Tesla', 'value': 'TSLA'}
        ],
        value='AAPL'
    ),
    dcc.Graph(id='live-graph'),
    dcc.Interval(
        id='interval-component',
        interval=100*1000,  # 10 seconds in milliseconds
        n_intervals=0
    )
])

def fetch_stock_data(symbol):
    api_key = 'HI7TGUC8NAMVHL4I'  # Replace with your Alpha Vantage API key
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval=1min&apikey={api_key}'
    response = requests.get(url)
    data = response.json()

    # Print the data for debugging if the key is missing
    if 'Time Series (1min)' not in data:
        print(f"Error in API response: {data}")  # For debugging, print the full response
        return None  # Return None if data is invalid

    return data

@app.callback(
    Output('live-graph', 'figure'),
    [Input('stock-dropdown', 'value'),
     Input('interval-component', 'n_intervals')]
)
def update_graph(selected_stock, n):
    data = fetch_stock_data(selected_stock)

    # Handle case when data is None (due to API errors)
    if data is None:
        return go.Figure()  # Return an empty figure if no valid data is fetched

    time_series = data.get('Time Series (1min)', {})

    if not time_series:
        return go.Figure()  # Return empty figure if no time series data is available

    # Prepare data for plotting
    times = list(time_series.keys())
    times.sort()  # Sort times in ascending order
    prices = [float(time_series[time]['1. open']) for time in times]

    # Create the figure
    figure = go.Figure()
    figure.add_trace(go.Scatter(x=times, y=prices, mode='lines+markers', name=selected_stock))
    figure.update_layout(title=f'Real-Time Stock Price for {selected_stock}', xaxis_title='Time', yaxis_title='Price (USD)')

    return figure

if __name__ == '__main__':
    app.run_server(debug=True)